### 5.3 ブロック付きメソッドの定義

今回はブロック付きメソッドとその定義の仕方について解説する。ブロック付きメソッドとは、メソッドにブロックと呼ばれる処理のまとまりを渡して、メソッド内でブロックを呼び出すことができるメソッドのことである。

```ruby
[メソッド名] (メソッド引数){|ブロック引数|
	処理...
}
```

では実際にブロック付きメソッドを定義するときに渡されたブロックを呼び出すにはどうすればいいのだろうか？　以下のrunメソッドを見てみよう。

In [2]:
def run
    yield
end

:run

In [3]:
run{ p 'Hello'}

"Hello"


"Hello"

このrunメソッドはブロック付きメソッドの定義の仕方を使って定義された、渡されたブロックを **そのまま実行する** メソッドである。runメソッド内のyieldメソッドは渡されたブロックを実行するメソッドである。ブロック引数を渡すにはそのままyieldに渡せばよい。

```ruby
def run
	yield(1, 2, 3)
end

run{|a, b, c|
	puts "#{a}, #{b}, #{c}"
}
#=> 1, 2, 3
```

ブロック付きメソッドの良いところは、メソッド内の処理の一部を呼び出し時に渡すことで、処理の抽象度を上げられることである。これは、次の章で習う配列でより詳しく説明される。

### 演習
ブロック付きメソッドを用いて、三つの整数の積を標準出力するようなプログラムを書け。

### チェックリスト
* ブロック付きメソッドのメリットは何か？